## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [0]:
import tensorflow as tf

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd3

In [0]:

tf.enable_eager_execution()
data = pd.read_csv('/content/drive/My Drive/ML/prices.csv')

### Check all columns in the dataset

In [22]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Drop columns `date` and  `symbol`

In [5]:
pdata = data.drop(['date','symbol'], axis=1)
pdata.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


In [6]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [17]:
nData = pdata.head(1000)
nData['volume'] = (nData['volume'] / 1000000)
nData.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,0.000002
1,125.239998,119.980003,119.940002,125.540001,0.000002
2,116.379997,114.949997,114.930000,119.739998,0.000002
3,115.480003,116.620003,113.500000,117.440002,0.000002
4,117.010002,114.970001,114.089996,117.330002,0.000001


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split



In [0]:
X = nData.drop('volume', axis=1)
y = nData['volume']
# X.head(5)
# y.loc[:5]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=7)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
arr_X_test = X_test.to_numpy(dtype="float32")
arr_X_train = X_train.to_numpy(dtype="float32")
arr_y_test = y_test.to_numpy(dtype="float32")
arr_y_train = y_test.to_numpy(dtype="float32")


### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import StandardScaler

In [0]:
# scaler = StandardScaler()
# scaler.fit(arr_X_test)

from sklearn.preprocessing import Normalizer

xTrain = Normalizer().fit(arr_X_train).transform(arr_X_train)
xTest = Normalizer().fit(arr_X_test).transform(arr_X_test)



## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
# Defineing Weights and Bias

W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

2.Define a function to calculate prediction

In [0]:
# y_pred = tf.add(tf.matmul(xTrain,W),b,name='output')

def pred_fn(X,W,b):
  return tf.add(tf.matmul(X,W),b,name='output')

3.Loss (Cost) Function [Mean square error]

In [0]:
# loss = tf.reduce_mean(tf.square(arr_y_train-y_pred),name='Loss')
# print(loss)

def loss_fn(y_train, y_pred):
  loss = tf.reduce_mean(tf.square(arr_y_train-y_pred),name='Loss')
  return loss

def 4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:

def train_fn(x,y, W,b, learn_rate):
  with tf.GradientTape() as t :
     t.watch([W,b])
     y_pred = pred_fn(x,W,b)
     loss = loss_fn(y,y_pred)
  
  dc_dw, dc_db = t.gradient(loss, [W, b])
  W = W - learn_rate * dc_dw
  b = b - learn_rate * dc_db
  return W,b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [65]:
for i in range (100):
  W,b = train_fn(xTrain,arr_y_train,W,b,0.01)
  y_pred = pred_fn(xTrain,W,b)
  train_loss = loss_fn(arr_y_train,y_pred)
  if i % 5 == 0:
    print('Training loss at step: ', i, ' is ', train_loss)

Training loss at step:  0  is  tf.Tensor(5.6916905e-11, shape=(), dtype=float32)
Training loss at step:  5  is  tf.Tensor(5.6915025e-11, shape=(), dtype=float32)
Training loss at step:  10  is  tf.Tensor(5.691378e-11, shape=(), dtype=float32)
Training loss at step:  15  is  tf.Tensor(5.6912947e-11, shape=(), dtype=float32)
Training loss at step:  20  is  tf.Tensor(5.6912395e-11, shape=(), dtype=float32)
Training loss at step:  25  is  tf.Tensor(5.691203e-11, shape=(), dtype=float32)
Training loss at step:  30  is  tf.Tensor(5.691178e-11, shape=(), dtype=float32)
Training loss at step:  35  is  tf.Tensor(5.691162e-11, shape=(), dtype=float32)
Training loss at step:  40  is  tf.Tensor(5.6911514e-11, shape=(), dtype=float32)
Training loss at step:  45  is  tf.Tensor(5.6911444e-11, shape=(), dtype=float32)
Training loss at step:  50  is  tf.Tensor(5.6911392e-11, shape=(), dtype=float32)
Training loss at step:  55  is  tf.Tensor(5.691136e-11, shape=(), dtype=float32)
Training loss at step: 

### Get the shapes and values of W and b

In [79]:
print("Shape and Value of W :")
print(W)

Shape and Value of W :
tf.Tensor(
[[1.1540517e-06]
 [1.1580388e-06]
 [1.1421811e-06]
 [1.1677614e-06]], shape=(4, 1), dtype=float32)


In [81]:
print("Shape and Value of b :")
print(b)

Shape and Value of b :
tf.Tensor([2.311143e-06], shape=(1,), dtype=float32)


### Model Prediction on 1st Examples in Test Dataset

In [83]:
y_pred = pred_fn(xTest,W,b)
test_loss = loss_fn(arr_y_test, y_pred)

print("Loss for test dataset: ", test_loss)

Loss for test dataset:  tf.Tensor(5.69113e-11, shape=(), dtype=float32)


## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:

iris = pd.read_csv('/content/drive/My Drive/ML/11_Iris.csv')

In [128]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:


X = iris.drop(['Id','Species'], axis=1)
# X.head()
y = pd.get_dummies(iris['Species'])

In [7]:
X.shape
y.shape

(150, 3)

### Splitting the data into feature set and target set

In [0]:

X = iris.drop(['Id','Species'], axis=1)
# X.head()
y = pd.get_dummies(iris['Species'])

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [36]:
model = tf.keras.models.Sequential()

model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(4,)),  # input shape required
  tf.keras.layers.Dense(3,activation=tf.nn.softmax,name='output')
])
#Compile the model - add Loss and Gradient Descent optimizer
model.compile(tf.keras.optimizers.SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X,y, epochs=5)


Epoch 1/5
150/150 [==============================] - 0s 368us/sample - loss: 1.2519 - acc: 0.3333
Epoch 2/5
150/150 [==============================] - 0s 94us/sample - loss: 1.0622 - acc: 0.3333
Epoch 3/5
150/150 [==============================] - 0s 86us/sample - loss: 0.9739 - acc: 0.2067
Epoch 4/5
150/150 [==============================] - 0s 81us/sample - loss: 0.9236 - acc: 0.1533
Epoch 5/5
150/150 [==============================] - 0s 93us/sample - loss: 0.8879 - acc: 0.3400


### Model Training 

In [40]:
# model.evaluate(X,y)
from sklearn.model_selection import train_test_split

X_train,X_test, y_train,y_test = train_test_split(X,y, test_size=0.30, random_state=7)
y_pred = model.fit(X_train,y_train,epochs=100)



Epoch 1/100
105/105 [==============================] - 0s 129us/sample - loss: 0.6512 - acc: 0.8476
Epoch 2/100
105/105 [==============================] - 0s 114us/sample - loss: 0.6407 - acc: 0.8286
Epoch 3/100
105/105 [==============================] - 0s 108us/sample - loss: 0.6318 - acc: 0.8286
Epoch 4/100
105/105 [==============================] - 0s 115us/sample - loss: 0.6248 - acc: 0.8000
Epoch 5/100
105/105 [==============================] - 0s 116us/sample - loss: 0.6148 - acc: 0.7810
Epoch 6/100
105/105 [==============================] - 0s 105us/sample - loss: 0.6066 - acc: 0.8381
Epoch 7/100
105/105 [==============================] - 0s 103us/sample - loss: 0.5993 - acc: 0.8476
Epoch 8/100
105/105 [==============================] - 0s 127us/sample - loss: 0.5915 - acc: 0.7524
Epoch 9/100
105/105 [==============================] - 0s 113us/sample - loss: 0.5854 - acc: 0.7238
Epoch 10/100
105/105 [==============================] - 0s 123us/sample - loss: 0.5785 - acc: 0.7714

### Model Prediction

In [41]:
model.evaluate(X_test, y_test)

45/45 [==============================] - 0s 714us/sample - loss: 0.3813 - acc: 0.9556


[0.38129538165198434, 0.95555556]

### Save the Model

In [0]:
model.save("iris_model.h5")

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?